In [1]:
!pip install -q autogluon

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.2 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
aiobotocore 2.12.1 requires botocore<1.34.52,>=1.34.41, but you have botocore 1.29.165 which is incompatible.
albumentations 1.4.0 requires scikit-image>=0.21.0, but you have scikit-image 0.20.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.5 which is incomp

In [2]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor,TabularDataset

warnings.filterwarnings('ignore')
seed = 27

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv', index_col='id')

In [4]:
train['Sex'] = train['Sex'].map({'M': 0, 'F': 1, 'I': 2})
test['Sex'] = test['Sex'].map({'M': 0, 'F': 1, 'I': 2})

train['Rings'] = np.log1p(train['Rings'])

In [5]:
_train, _val = train_test_split(train, test_size=0.15, random_state=seed)

In [6]:
_train = TabularDataset(_train)
_val = TabularDataset(_val)

test = TabularDataset(test)

In [7]:
predictor = TabularPredictor(
    label="Rings",
    eval_metric='root_mean_squared_error',
    problem_type="regression",
    verbosity=2
).fit(
    train_data=_train,
    presets="best_quality",
    time_limit=3600 * 8
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240405_050318"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 28800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240405_050318/ds_sub_fit/sub_fit_ho.
2024-04-05 05:03:18,876	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning A

In [8]:
score = -predictor.evaluate(_val, silent=True)['root_mean_squared_error']

INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [9]:
print(f"Validation RMSE Score: {score}")

Validation RMSE Score: 0.14707108662471752


### Why change the sign?
- Take a look at this issue on GitHub: https://github.com/autogluon/autogluon/issues/524

In [10]:
fi = predictor.feature_importance(data=_val, silent=True)
lb = predictor.leaderboard(_val, silent=True)
lb['score_test'] = lb['score_test'].apply(lambda x: -x)
lb['score_val'] = lb['score_val'].apply(lambda x: -x)

INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighb

In [11]:
lb.style.background_gradient(subset=['score_val', 'score_test'], cmap='RdYlGn_r')

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.147071,0.147627,root_mean_squared_error,585.626190,2699.787648,17417.987465,0.007030,0.002719,3.055892,3,True,73
1,NeuralNetFastAI_r191_BAG_L2,0.147097,0.147797,root_mean_squared_error,573.684459,2668.408614,14876.425125,3.454546,3.862144,1000.184548,2,True,59
2,NeuralNetFastAI_BAG_L2,0.147115,0.147771,root_mean_squared_error,572.645803,2668.094552,14242.276825,2.415891,3.548083,366.036248,2,True,52
3,NeuralNetFastAI_r102_BAG_L2,0.147126,0.147778,root_mean_squared_error,570.966428,2665.674092,14008.485849,0.736515,1.127622,132.245272,2,True,66
4,LightGBM_r96_BAG_L2,0.147165,0.147929,root_mean_squared_error,578.010436,2700.707196,14139.757705,7.780523,36.160726,263.517128,2,True,61
5,LightGBMXT_BAG_L2,0.147224,0.148014,root_mean_squared_error,571.690686,2670.681839,13983.214944,1.460774,6.135369,106.974367,2,True,47
6,LightGBM_r131_BAG_L2,0.147266,0.148017,root_mean_squared_error,572.493240,2674.249630,14059.092431,2.263327,9.703161,182.851854,2,True,58
7,LightGBMLarge_BAG_L2,0.147272,0.148382,root_mean_squared_error,571.388854,2668.728297,14011.217075,1.158941,4.181827,134.976498,2,True,55
8,LightGBM_BAG_L2,0.147291,0.147998,root_mean_squared_error,570.654924,2666.233974,13945.594712,0.425011,1.687504,69.354135,2,True,48
9,CatBoost_r9_BAG_L2,0.147304,0.147920,root_mean_squared_error,570.806015,2665.369050,14101.230320,0.576102,0.822581,224.989743,2,True,60


In [12]:
fi.style.background_gradient(cmap='RdYlGn')

,importance,stddev,p_value,n,p99_high,p99_low
Shell weight,0.142379,0.001667,0.000000,5,0.145812,0.138946
Whole weight,0.136993,0.001188,0.000000,5,0.139438,0.134547
Whole weight.1,0.103392,0.003607,0.000000,5,0.110819,0.095965
Whole weight.2,0.041217,0.000996,0.000000,5,0.043269,0.039166
Height,0.011334,0.000716,0.000002,5,0.012808,0.009859
Diameter,0.011009,0.000536,0.000001,5,0.012112,0.009905
Length,0.008896,0.000945,0.000015,5,0.010842,0.006951
Sex,0.004627,0.000333,0.000003,5,0.005313,0.003940


In [13]:
preds = predictor.predict(test)
preds = np.expm1(preds.values)

INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [14]:
submission = pd.DataFrame({'id': test.index, 'Rings': preds})
submission.to_csv(f'autogluon-{score:.6f}.csv', index=False)
submission.head()

,id,Rings
0,90615,9.862020
1,90616,9.701034
2,90617,9.772388
3,90618,10.478710
4,90619,7.558111
